In [1]:
import arcpy
import numpy as np
import netCDF4 as nc 
from arcpy.sa import *
import pandas as pd

In [2]:
def windcal(u,v):
    """
    # windcal can calculate the overall wind direction based on
    # Input u and v wind component
    # the out put wind direction represent the wind is blowing to, if you want find wind is coming from, you need to uncomment lats step
    """

    ws = (u**2 + v**2)**0.5
    wd = np.arctan2(v,u)
    wd_ang = wd *180/np.pi
    wd_ang = wd_ang + 180

    return wd_ang,ws

def geo_idx(dd, dd_array):
    """
     - dd - the decimal degree (latitude or longitude)
     - dd_array - the list of decimal degrees to search.
     search for nearest decimal degree in an array of decimal degrees and return the index.
     np.argmin returns the indices of minium value along an axis.
     so subtract dd from all values in dd_array, take absolute value and find index of minium.
   """
    geo_idx = (np.abs(dd_array - dd)).argmin()
    return geo_idx


In [3]:
def extract_wind(comp,wind_path,source,lat,lon,t):
    if comp: 
        wind_data = nc.Dataset(wind_path,'r')
        lats = wind_data.variables['latitude'][:]
        lons = wind_data.variables['longitude'][:]
        u = wind_data.variables['u10'][t,:,:]
        v = wind_data.variables['v10'][t,:,:]
        wind_data.close()
        wd,ws = windcal(u,v)
        
        # create wind direction and wind speed list to record wind speed & direction for all emission sources 
        site_wd = []
        site_ws = []
        
        # create search cursor to read source feature class/ shapefile 
        T1 = ['OBJECTID','SHAPE@',lat,lon]
        c1 =arcpy.da.SearchCursor(source,T1)
        for row in c1:
            s_id = row[0]
            s_geometry = row[1]
            
            if row[1] == None:
                pass 
            else:
                in_lat = row[2]
                in_lon = row[3]
                # since lons are 0 thru 360, convert to -180 thru 180
                converted_lons = lons - ( lons.astype(np.int32) / 180) * 360
                # get cell of facility
                lat_idx = geo_idx(in_lat, lats)
                lon_idx = geo_idx(in_lon, converted_lons)

                #extract winddirection and wind speed from that cell
                d = wd[lat_idx,lon_idx]
                s = ws[lat_idx,lon_idx]

                site_wd.append(d)
                site_ws.append(s)
            
        
        # create attributes/fields to record wind speed & direction 
        inFeatures = source
        # 1. create wind speed attribute
        fieldName = "windspeed"
        field_type = "DOUBLE"
        arcpy.AddField_management(inFeatures, fieldName,field_type,
                                  field_is_nullable="NULLABLE")
        
        # 2. create wind direction attribute 
        fieldName = "winddir"
        field_type = "DOUBLE"
        arcpy.AddField_management(inFeatures, fieldName,field_type,
                              field_is_nullable="NULLABLE")
        
        # insert extracted ERA wind speed & direction into the source feature class 
        # 1. update wind speed
        fn = ["windspeed","winddir"]
        with arcpy.da.UpdateCursor(inFeatures,fn) as cursor:
            index = 0
            for row in cursor: 
                row[0] = site_ws[index]
                row[1] = site_wd[index]
                cursor.updateRow(row)
                index = index + 1 
        del cursor
        print ("Extracting wind....")
        return fn[1]
    
    else: 
        
        # if you are using rater data 
        # Check out the ArcGIS Spatial Analyst extension license
        arcpy.CheckOutExtension("Spatial")
        fn = wind_path
        # Execute ExtractValuesToPoints
        ExtractMultiValuesToPoints(source, wind_path)
        print ("Extracting wind....")
        return fn  

In [4]:
def def_dist(source,dist): 
    # Update the distance field 
    fieldName = "Distance"
    field_type = "DOUBLE"
    arcpy.AddField_management(source, fieldName,field_type,
                                  field_is_nullable="NULLABLE")
    # update maximum downwind distance (length of wind line)
    with arcpy.da.UpdateCursor(source,fieldName) as cursor:
        for row in cursor:
            # OTM 33A downwind distance 200 meters
            cursor.updateRow([dist])
    del cursor
    
    print ("defining distance....")
    
    return fieldName

In [5]:
def find_n_DPRIP (source,DPRIPs): 
    # build 1st search cursor for source feature class
    F1 = ['OBJECTID','SHAPE@','SHAPE@X','SHAPE@Y']
    # list used to record nearest downiwnd distance
    ID_list =[]
    D_wind = []
    Int_x =[]
    Int_y =[]
    SX = []
    SY = []
    c1 =arcpy.da.SearchCursor(source,F1)
    i = 0 
    for row1 in c1:
        s_id = row1[0]
        s_geometry = row1[1]
        s_lon = row1[2]
        s_lat = row1[3]
        # temporary list used to record distance 
        distlist = []
        xlist = [] 
        ylist = []
        F2 = ['FID_bearline','SHAPE@','SHAPE@X','SHAPE@Y']
        c2 = arcpy.da.SearchCursor(DPRIPs,F2)
        for row2 in c2:
            i_id  = row2[0]
            i_geometry = row2[1]
            i_lon = row2[2]
            i_lat = row2[3]
            
            if i_id == s_id:
                dis = s_geometry.distanceTo(i_geometry)
                distlist.append(dis)
                xlist.append(i_lon)
                ylist.append(i_lat)
            
        if len(distlist) == 0:
            fac_dist = -999
            nt_x =-999
            nt_y = -999
        else:
            fac_dist = min(distlist)
            near_ind = distlist.index(min(distlist)) 
            
            nt_x = xlist[near_ind]
            nt_y = ylist[near_ind]
        
        Int_x.append(nt_x)
        Int_y.append(nt_y)
        D_wind.append(fac_dist)
        SX.append(s_lon)
        SY.append(s_lat)
        ID_list.append(s_id)
        
        print (i)
        i += 1 
        
    print ("found all nearest DPRIPs")    
    return ID_list,D_wind,Int_x,Int_y,SX,SY



In [6]:
def def_width(source,width): 
    # Update the plume width field 
    fieldName = "Width"
    field_type = "DOUBLE"
    arcpy.AddField_management(source, fieldName,field_type,
                                  field_is_nullable="NULLABLE")
    # update cursor 
    with arcpy.da.UpdateCursor(source,fieldName) as cursor:
        index = 0 
        for row in cursor:
            # update width 
            row[0] = width[index]
            cursor.updateRow(row)
            index = index + 1
    del cursor
    
    print ("Defining width of plume....")
    
    return fieldName

In [8]:
##################################################### Workflow ######################################################

###### Part 1 #####
# define arcpy environment
arcpy.env.workspace =r"D:\DPRIE_Test\DPRIE.gdb"
source = 'source'
wind = "WindDirection"

# extract wind (I used raster data)
fn = extract_wind(False,wind,source,0,0,0)
# defined distance
dn = def_dist(source,500)
# create virtual downiwnd trajectory
# re-define arcpy environemnt
arcpy.env.workspace =r"D:\DPRIE_Test\DPRIE.gdb"
road = "road"
DPRIPs = "DPRIP"
bl = "bearline"
bl_path = arcpy.BearingDistanceToLine_management(source, bl, 'X', 'Y',
                                       dn,"METERS",fn, "DEGREES",'GEODESIC')
print ("creating virtual downiwnd trajectory....")
# find all DPRIPs
dps_path = arcpy.Intersect_analysis ([road, bl],DPRIPs, "", "", 'POINT')

# find nearest DPRIPs for each source, calculate nearest downiwnd distance,
# record coordinates of nearest DPRIPs
print ("calculating DPRIPs....")
ID_list,D_wind,Int_x,Int_y,SX,SY = find_n_DPRIP(source,DPRIPs)

# output first part of results
d = {"source_X":SX,"source_Y":SY,"source_ID":ID_list,
    "Downwind_dist":D_wind,"p_x":Int_x,"p_y":Int_y}

df = pd.DataFrame(data=d,columns = ["source_ID","source_X","source_Y",
                                   "p_x","p_y","Downwind_dist"])
ndf = df[df.Downwind_dist!= -999]

allsource = r"D:\DPRIE_Test\allsource.csv"
df.to_csv(allsource)
DPRIP_t = r"D:\DPRIE_Test\eff_source.csv"
ndf.to_csv(DPRIP_t)

print ("Part I finished!!!Outputing results....")

###### Part 2 #####
# re-define arcpy environemnt
arcpy.env.workspace =r"D:\DPRIE_Test\DPRIE.gdb"
out_lines = "nsri_line"
input_table = DPRIP_t
# define the spatial reference
sr = arcpy.SpatialReference(54002) #WKID of equidistant cylinderical

# create virtual line
arcpy.XYToLine_management(input_table,out_lines,
                         "source_X","source_Y","p_x",
                         "p_y","GEODESIC","source_ID",sr)

# calculate width of plume for each source & nearest DPRIP
pl = ndf.Downwind_dist
# calculate plume width 50% of plume length
pw = np.array(pl)/4
pw = np.round(pw)
# define the width of plume
wn = def_width(out_lines,pw)
# create virtual plume buffer
buffer = "line_buffer"
arcpy.Buffer_analysis(out_lines, buffer, wn, "Full", "FLAT")
print ("We got Plume")
# zonal statistics -> find dominant landcover type and output landcover type info
arcpy.env.workspace =r"D:\DPRIE_Test\DPRIE.gdb"
ras = r"D:\DPRIE_Test\DPRIE.gdb\landcover"
csv_path = r"D:\DPRIE_Test"
csv_name = "lancover_test.csv"
dbf_name = r"D:\DPRIE_Test\lc_test.dbf"
zoneField = "OID"
outTable = dbf_name
stat = "MAJORITY"
ZonalStatisticsAsTable(buffer, zoneField,ras,dbf_name, "DATA", stat)
arcpy.TableToTable_conversion(dbf_name, csv_path, csv_name)
# zonal statistics -> find the elevation range
ras = r"D:\DPRIE_Test\DPRIE.gdb\DEM"
csv_path = r"D:\DPRIE_Test"
csv_name = "DEM_test.csv"
dbf_name = r"D:\DPRIE_Test\DEM_test.dbf"
zoneField = "OID"
outTable = dbf_name
stat = "Range"
ZonalStatisticsAsTable(buffer, zoneField,ras,dbf_name, "DATA", stat)
arcpy.TableToTable_conversion(dbf_name, csv_path, csv_name)
print ("Part II finished!!!!")

Extracting wind....
defining distance....
creating virtual downiwnd trajectory....
calculating DPRIPs....
0
1
2
3
4
5
6
7
8
9
10
found all nearest DPRIPs
Part I finished!!!Outputing results....
Defining width of plume....
We got Plume
Part II finished!!!!


Doing zonal statistics....
